<a href="https://colab.research.google.com/github/Blake2912/Vahini-2.0/blob/simulation-main/simulation/Water_Level_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install osmnx networkx shapely ipyleaflet ipywidgets -q

from time import sleep
import urllib.request
import osmnx as ox
import networkx as nx
import ipyleaflet as ipl
from time import sleep
import pickle
import shapely
from shapely.geometry import Polygon
from ipyleaflet import Map, Marker, FullScreenControl, AwesomeIcon
from ipywidgets import interact
from time import sleep

url = "https://github.com/Blake2912/Vahini-2.0/raw/simulation-main/output/Trees_Final.pkl"
urllib.request.urlretrieve(url, "Trees_Final.pkl")
filename = "/content/Trees_Final.pkl"

with open(filename, "rb") as f:
    G = pickle.load(f)

In [10]:
center = (12.96682, 77.71231)
m1 = ipl.Map(center=center, basemap=ipl.basemaps.CartoDB.Positron, zoom=18)
m1.layout.height = '800px'
import re
# Setting up trees on the path
pattern = r't\-[0-9]+'
for point_name in list(G.nodes()):
    if re.match(pattern, point_name):
        # Get the latitude and longitude of the node
        latitude = G.nodes[point_name]['y']
        longitude = G.nodes[point_name]['x']
        # Draw circle for the point
        circle = ipl.Circle(
            location=(latitude, longitude),
            radius=2,
            color="tomato",
        )
        m1.add_layer(circle)
# Adding markers in the interactive map
icon1 = AwesomeIcon(
    name='fa-truck',
    marker_color='green',
    icon_color='black',
    spin=False
)

marker = ipl.Marker(icon=icon1, location=[G.nodes['basic_science']['y'],G.nodes['basic_science']['x']])
m1.add_layer(marker)

In [12]:
display(m1)
# Finding optimal path when vehicle running low on water

current_vehicle_location = "t-4" # Let's assume its at tree 4
base_location = "basic_science" # This is the charging station

water_level = 15 # Let us take the water level in percentage
water_threshold = 20 # Threshold value for water level

shortest_route = nx.shortest_path(G, current_vehicle_location, base_location, weight='length')
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]

if water_level <= water_threshold:
    # The vehicle needs to move to base
    step = 1.5
    marker.location = route_coords[0]
    marker.color = "yellow"

    # Adding the path from the given nodes
    shortest_path = ipl.Polyline(
        locations=route_coords,
        color="blue",
        fill=False
    )
    m1.add_layer(shortest_path)
    # Move the vehicle
    for point in route_coords:
        marker.location = point
        sleep(step)

else:
    print("Vehicle can continue with the given water level")

Map(bottom=31116739.0, center=[12.96682, 77.71231], controls=(ZoomControl(options=['position', 'zoom_in_text',…